In [1]:
#Set-up files

#Start parallelization process
#addprocs()

#@everywhere 

using JuMP, DataFrames, Gurobi, Queryverse

┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1186
ERROR: LoadError: LoadError: syntax: invalid "::" syntax
Stacktrace:
 [1] include at .\boot.jl:317 [inlined]
 [2] include_relative(::Module, ::String) at .\loading.jl:1038
 [3] include at .\sysimg.jl:29 [inlined]
 [4] include(::String) at C:\Users\Will\.julia\packages\JuMP\6aWuF\src\JuMP.jl:13
 [5] top-level scope at none:0
 [6] include at .\boot.jl:317 [inlined]
 [7] include_relative(::Module, ::String) at .\loading.jl:1038
 [8] include(::Module, ::String) at .\sysimg.jl:29
 [9] top-level scope at none:2
 [10] eval at .\boot.jl:319 [inlined]
 [11] eval(::Expr) at .\client.jl:389
 [12] top-level scope at .\none:3
in expression starting at C:\Users\Will\.julia\packages\JuMP\6aWuF\src\JuMPContainer.jl:18
in expression starting at C:\Users\Will\.julia\packages\JuMP\6aWuF\src\JuMP.jl:55


ErrorException: Failed to precompile JuMP [4076af6c-e467-56ae-b986-b466b2749572] to C:\Users\Will\.julia\compiled\v1.0\JuMP\DmXqY.ji.

In [ ]:
# Set-working directory
#DIR = "C:\\Users\\will-\\GoogleDrive\\UCBerkeley\\Research\\Papers\\2018 Off-grid\\"
DIR = "C:\\Users\\Will\\GoogleDrive\\UCBerkeley\\Research\\Papers\\2018 Off-grid\\"
OUT = "out"
INPUT = "in"

In [ ]:
##### CREATE MODEL RUN ######
# Set constants
LOAD_SHED = [0.25, 0.05, 0]
BAT_COST = 500 # $/kWh
PV_COST = 3000 # $/kW
BAT_EFF = 0.92
#annual rates
int_rate = 0.06 # percentage interest rate
bat_life = 10 # years
sol_life = 25 # years
BAT_RATE = int_rate / (1 - (1+int_rate)^(-bat_life))
PV_RATE = int_rate / (1 - (1+int_rate)^(-sol_life))
NEM = 0.12
VOS = 0.03

# identify geography IDs to loop through
ID_G = load(DIR * INPUT * "\\id.csv") |> DataFrame



In [ ]:
#Run Model
# Create parallelization
timing = @elapsed outputs = pmap(1:length(LOAD_SHED)*length(ID_G[1])) do i #
    solar_opt(ID_G, LOAD_SHED, BAT_COST, BAT_RATE, BAT_EFF, PV_COST, PV_RATE, NEM, DIR, INPUT, i) 
end

In [ ]:
# Collect results

#write status
array = outputs |> @map(x->x[1]) |> collect 
df = DataFrame(status = array) 
save(DIR * OUT * "\\status_v5.csv", df) 

#write results table
array = outputs |> @map(x->x[2]) |> collect
for i = 2:length(array) 
    df = array[1]
    df = append!(df, array[i])
end
save(DIR * OUT * "\\results_v5.csv", df)

#write outcome table
array = outputs |> @map(x->x[3]) |> collect
for i = 2:length(array) 
    df = array[1]
    df = append!(df, array[i])
end
save(DIR * OUT * "\\outcome_v5.csv", df)